# INFORME TÉCNICO: VRP con Capacidad Aumentada (80 kg)

## 1. Introducción y Cambio de Escenario

En este experimento se aborda el **Problema de Enrutamiento de Vehículos con Capacidad (CVRP)** bajo una nueva restricción operativa. A diferencia del escenario anterior, la capacidad de la flota se ha incrementado a **80 kg**.

**Objetivo del Experimento:**
Analizar cómo el aumento de capacidad afecta la **factibilidad de los clientes** y la **eficiencia de las rutas**. Específicamente, se busca integrar al cliente con mayor demanda (C5), que anteriormente era excluido, y minimizar la distancia total recorrida sirviendo al 100% de la cartera de clientes.

## 2. Análisis de Datos y Restricciones

### 2.1. Perfil de Demanda
Las demandas calculadas para los 5 clientes son:

| Cliente | Demanda (kg) | Estado (Cap. 60) | Estado (Cap. 80) |
| :---: | :---: | :---: | :---: |
| C1 | 18.6 | Válido | **Válido** |
| C2 | 22.8 | Válido | **Válido** |
| C3 | 48.0 | Válido | **Válido** |
| C4 | 60.0 | Válido | **Válido** |
| C5 | **77.7** | *Inválido* | **Válido** |

### 2.2. Impacto de la Nueva Capacidad
El cambio a 80 kg tiene un impacto crítico:
1.  **Inclusión Total:** El Cliente 5 (77.7 kg) ahora cumple con la restricción ($77.7 \le 80$). El Algoritmo Genético (AG) ahora debe optimizar una ruta para **5 clientes** en lugar de 4.
2.  **Saturación de Carga:** El Cliente 5 ocupará el 97% de la capacidad de un camión, forzando prácticamente un viaje exclusivo para él (ya que solo sobran 2.3 kg y ningún otro cliente tiene una demanda tan pequeña).
3.  **Combinatoria:** La capacidad extra permite nuevas agrupaciones. Por ejemplo, C4 (60 kg) ahora deja 20 kg libres, lo que permite agruparlo con C1 (18.6 kg), algo imposible con la capacidad de 60.

## 3. Metodología (Algoritmo Genético)

Se mantiene la configuración del AG para garantizar la comparabilidad:
* **Representación:** Permutación de índices de todos los clientes válidos.
* **Evaluación (Fitness):** Distancia Euclídea total.
* **Decodificación de Rutas:** Heurística de "Siguiente Ajuste" (llenar el vehículo hasta que el siguiente cliente no quepa, volver al depósito y empezar nueva ruta).
* **Parámetros:** 200 Generaciones, Población de 50 individuos.

--- 
### Implementación del Código
*A continuación se presentan las celdas de código para la ejecución del experimento.*

In [1]:
import random
import math
import matplotlib.pyplot as plt

# Datos del problema
depot = [20, 120]
clientes = [
    [35, 115], [50, 140], [70, 100], [40, 80], [25, 60]
]

pesos = [1.2, 3.8, 7.5, 0.9, 15.4, 12.1, 4.3, 19.7, 8.6, 2.5]

pedidos = [
    [(3, 2), (1, 3)],
    [(2, 6)],
    [(7, 4), (5, 2)],
    [(3, 8)],
    [(6, 5), (9, 2)]
]

In [3]:
# 3. Cálculo de demandas y Definición de Capacidad
demandas = []
for pedido in pedidos:
    total = 0
    for item, cant in pedido:
        total += pesos[item - 1] * cant
    demandas.append(total)

# AQUI ESTA EL CAMBIO CLAVE: Capacidad aumentada a 80
capacidad = 80

print('Demandas calculadas:', [round(d,1) for d in demandas])
print(f'Capacidad del vehículo: {capacidad} kg')

In [4]:
# 4. Filtrado de clientes
clientes_validos = []
demandas_validas = []
clientes_invalidos = []
demandas_invalidas = []

for i in range(len(clientes)):
    if demandas[i] <= capacidad:
        clientes_validos.append(clientes[i])
        demandas_validas.append(demandas[i])
    else:
        clientes_invalidos.append(clientes[i])
        demandas_invalidas.append(demandas[i])

print(f'Clientes válidos: {len(clientes_validos)} (Todos incluidos)')
print(f'Clientes inválidos: {len(clientes_invalidos)}')

In [5]:
# 5. Funciones del AG (Distancia, Fitness, Cruce, Mutación)
def distancia(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def dividir_en_rutas(ruta):
    rutas = []
    carga = 0
    actual = []
    for c in ruta:
        if carga + demandas_validas[c] <= capacidad:
            actual.append(c)
            carga += demandas_validas[c]
        else:
            rutas.append(actual)
            actual = [c]
            carga = demandas_validas[c]
    if actual:
        rutas.append(actual)
    return rutas

def distancia_total(ruta):
    rutas = dividir_en_rutas(ruta)
    total = 0
    for r in rutas:
        pos = depot
        for c in r:
            total += distancia(pos, clientes_validos[c])
            pos = clientes_validos[c]
        total += distancia(pos, depot)
    return total

def fitness(ruta):
    return distancia_total(ruta)

def crear_poblacion(n, n_clientes):
    poblacion = []
    base = list(range(n_clientes))
    for _ in range(n):
        r = base[:]
        random.shuffle(r)
        poblacion.append(r)
    return poblacion

def seleccion(poblacion, fitnesses):
    i1, i2 = random.sample(range(len(poblacion)), 2)
    return poblacion[i1][:] if fitnesses[i1] < fitnesses[i2] else poblacion[i2][:]

def cruce(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    hijo = [-1] * len(p1)
    hijo[a:b] = p1[a:b]
    pos = b
    for x in p2:
        if x not in hijo:
            if pos >= len(p1): pos = 0
            hijo[pos] = x
            pos += 1
    return hijo

def mutacion(ruta, prob=0.1):
    if random.random() < prob:
        i, j = random.sample(range(len(ruta)), 2)
        ruta[i], ruta[j] = ruta[j], ruta[i]

def algoritmo_genetico(n_generaciones=200, tam_pob=50):
    poblacion = crear_poblacion(tam_pob, len(clientes_validos))
    fitnesses = [fitness(r) for r in poblacion]

    for _ in range(n_generaciones):
        nueva = []
        for _ in range(tam_pob):
            p1 = seleccion(poblacion, fitnesses)
            p2 = seleccion(poblacion, fitnesses)
            hijo = cruce(p1, p2)
            mutacion(hijo, 0.1)
            nueva.append(hijo)
        poblacion = nueva
        fitnesses = [fitness(r) for r in poblacion]

    mejor = min(range(tam_pob), key=lambda i: fitnesses[i])
    return poblacion[mejor], fitnesses[mejor]

## 4. Ejecución y Análisis de Resultados

Al ejecutar el algoritmo con la nueva capacidad, se observa un comportamiento de agrupación muy eficiente. A continuación se muestran los resultados:

In [8]:
mejor_ruta, mejor_valor = algoritmo_genetico()

print('Mejor secuencia (indices):', mejor_ruta)
print('Distancia total:', round(mejor_valor, 2))

print("\n--- Detalle de Rutas Generadas ---")
rutas = dividir_en_rutas(mejor_ruta)
for i, r in enumerate(rutas):
    carga = sum(demandas_validas[c] for c in r)
    uso_capacidad = (carga / capacidad) * 100
    print(f'Viaje {i+1}: Clientes {r} -> Carga: {round(carga,1)} kg ({round(uso_capacidad, 1)}% ocupación)')

In [10]:
def plot_rutas(rutas):
    plt.figure(figsize=(8, 8))
    # Dibujar clientes
    for i, (x, y) in enumerate(clientes_validos):
        plt.scatter(x, y, c='blue', zorder=5)
        plt.text(x+1, y+1, f'C{i+1} ({round(demandas_validas[i],1)}kg)', fontsize=9)

    # Dibujar depósito
    plt.scatter(depot[0], depot[1], c='red', s=100, marker='s', label='Depósito', zorder=5)

    colores = ['green', 'orange', 'purple', 'cyan']
    
    for i, r in enumerate(rutas):
        puntos = [depot] + [clientes_validos[c] for c in r] + [depot]
        xs = [p[0] for p in puntos]
        ys = [p[1] for p in puntos]
        carga = sum(demandas_validas[c] for c in r)
        plt.plot(xs, ys, '-o', color=colores[i % len(colores)], label=f'Viaje {i+1} ({round(carga,1)}kg)')

    plt.title(f"Rutas Óptimas (Capacidad {capacidad}kg)")
    plt.xlabel("Coordenada X")
    plt.ylabel("Coordenada Y")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

plot_rutas(rutas)

## 5. Conclusiones Finales del Experimento (Capacidad 80 kg)

El aumento de la capacidad del vehículo de 60 kg a **80 kg** ha transformado radicalmente la solución logística. Las principales conclusiones son:

1.  **Viabilidad Total (C5 Incluido):**
    * El factor más crítico es la inclusión del **Cliente 5** (77.7 kg). Al aumentar la capacidad, este cliente, que antes representaba una venta perdida o una excepción logística, ahora entra dentro de los parámetros operativos estándar.
    
2.  **Optimización de Carga (High Filling Rate):**
    * Se observa una **saturación de flota** mucho más eficiente. 
    * La combinación del **Cliente 4** (60 kg) con el **Cliente 1** (18.6 kg) suma **78.6 kg**. Esto representa una ocupación del **98.2%**, un escenario ideal en logística que era matemáticamente imposible con la restricción de 60 kg.
    * El Cliente 5 ocupa por sí solo el **97.1%** de un vehículo, justificando un viaje dedicado.

3.  **Consolidación de Rutas:**
    * A diferencia del escenario anterior donde los vehículos viajaban con cargas parciales (ej. 48 kg de 60 = 80%), aquí se logra mover más mercancía (todos los clientes) con un uso casi total del espacio disponible en cada viaje.
    * Esto demuestra que un aumento del **33% en la capacidad** (de 60 a 80) no solo permite atender clientes más grandes, sino que mejora drásticamente la eficiencia combinatoria del resto de la flota.